In [6]:
from huggingface_hub import login

login()

In [15]:
from transformers import AutoModelForVision2Seq, AutoProcessor, AutoTokenizer

model = AutoModelForVision2Seq.from_pretrained(
    "callmeeric5/Qwen3B-Invoice-Receipt",
    device_map="cuda",
    torch_dtype="auto"
)
tokenizer = AutoTokenizer.from_pretrained("callmeeric5/Qwen-3B-Invoice-Receipt")
processor = AutoProcessor.from_pretrained("callmeeric5/Qwen-3B-Invoice-Receipt")
model.eval()

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Qwen2_5_VLForConditionalGeneration(
  (model): Qwen2_5_VLModel(
    (visual): Qwen2_5_VisionTransformerPretrainedModel(
      (patch_embed): Qwen2_5_VisionPatchEmbed(
        (proj): Conv3d(3, 1280, kernel_size=(2, 14, 14), stride=(2, 14, 14), bias=False)
      )
      (rotary_pos_emb): Qwen2_5_VisionRotaryEmbedding()
      (blocks): ModuleList(
        (0-31): 32 x Qwen2_5_VLVisionBlock(
          (norm1): Qwen2RMSNorm((1280,), eps=1e-06)
          (norm2): Qwen2RMSNorm((1280,), eps=1e-06)
          (attn): Qwen2_5_VLVisionAttention(
            (qkv): Linear(in_features=1280, out_features=3840, bias=True)
            (proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (mlp): Qwen2_5_VLMLP(
            (gate_proj): Linear(in_features=1280, out_features=3420, bias=True)
            (up_proj): Linear(in_features=1280, out_features=3420, bias=True)
            (down_proj): Linear(in_features=3420, out_features=1280, bias=True)
            (act_fn): SiLU()

In [11]:
import json

schema_dict = {
    "document_type": "",
    "store_name": "",
    "store_address": "",
    "store_phone": "",
    "date": "",
    "time": "",
    "invoice_no": "",
    "invoice_date": "",
    "seller": "",
    "client": "",
    "seller_tax_id": "",
    "client_tax_id": "",
    "iban": "",
    "items": [
        {
            "item_name": "",
            "item_desc": "",
            "item_key": "",
            "item_quantity": "",
            "item_net_price": "",
            "item_value": "",
            "item_net_worth": "",
            "item_vat": "",
            "item_gross_worth": ""
        }
    ],
    "subtotal": "",
    "tax": "",
    "tips": "",
    "total": "",
    "total_net_worth": "",
    "total_vat": "",
    "total_gross_worth": ""
}

instruction = f"""
You are specialized in extracting information from invoices and receipts.
Your task is to extract the information from the provided document image in the following valid JSON format.
If any key is missing, leave it as an empty string.
Do NOT wrap keys in 'key':'value' pairs — output exactly this JSON structure:

{json.dumps(schema_dict, indent=2)}

Fill only the keys that are available.
"""

In [14]:
# ✅ 3️⃣ Load your test image
from PIL import Image
import torch
image = Image.open("test.jpg").convert("RGB")

# Add code to resize the image
max_size = 800
width, height = image.size
if max(width, height) > max_size:
    ratio = max_size / max(width, height)
    new_width = int(width * ratio)
    new_height = int(height * ratio)
    image = image.resize((new_width, new_height), Image.Resampling.LANCZOS)


# -------------------------------------------------------------
# ✅ 4️⃣ Create prompt exactly like training
messages = [
    {"role": "user", "content": [
        {"type": "text", "text": instruction},
        {"type": "image_url", "image_url": {"url": None}}
    ]}
]

prompt = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=False  # ✅ force plain string!
)

# -------------------------------------------------------------
# ✅ 5️⃣ Process input
inputs = processor(
    text=prompt,
    images=image,
    return_tensors="pt",
    truncation=False
)

# ✅ Safe move to CUDA & dtype
for k in inputs:
    if isinstance(inputs[k], torch.Tensor):
        inputs[k] = inputs[k].to("cuda")
        if torch.is_floating_point(inputs[k]):
            inputs[k] = inputs[k].to(model.dtype)

# -------------------------------------------------------------
# ✅ 6️⃣ Generate
from transformers import TextStreamer

streamer = TextStreamer(tokenizer, skip_prompt=True)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        streamer=streamer,
        max_new_tokens=512
    )

# ✅ If you want raw text back:
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\n\n--- ✅ Final JSON output ---\n")
print(result)

{"document_type": "receipt", "store_name": "CheaneyShoesLtd", "store_address": "London JermynStreet 21JermynSt", "store_phone": "02072872996,", "date": "15Jan19", "time": "12:11:44GMT", "invoice_no": "", "invoice_date": "", "seller": "", "client": "", "seller_tax_id": "", "client_tax_id": "", "iban": "", "items": [{"item_name": "SHOEFILLERS-CEDARWOODAO61OCHY", "item_desc": "", "item_key": "", "item_quantity": "1", "item_net_price": "", "item_value": "£350.00", "item_net_worth": "", "item_vat": "", "item_gross_worth": ""}, {"item_name": "ARTHURIIIBLACKCalf", "item_desc": "", "item_key": "", "item_quantity": "1", "item_net_price": "", "item_value": "£60.00", "item_net_worth": "", "item_vat": "", "item_gross_worth": ""}], "subtotal": "£341.67", "tax": "£68.33", "tips": "", "total": "£410.00 £410.00", "total_net_worth": "", "total_vat": "", "total_gross_worth": ""}<|im_end|>


--- ✅ Final JSON output ---

system
You are a helpful assistant.
user

You are specialized in extracting informati